In [1]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
LIMIT = 1  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers_df = pd.read_csv(TICKERS_URL)
tickers = tickers_df['Symbol'].tolist()

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]
    return {}

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers[150:230]):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income = get_fmp_data("income-statement", symbol)
    balance = get_fmp_data("balance-sheet-statement", symbol)
    cashflow = get_fmp_data("cash-flow-statement", symbol)

    if income or balance or cashflow:
        merged = {
            "symbol": symbol,
            "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
            **income,
            **{f"balance_{k}": v for k, v in balance.items()},
            **{f"cashflow_{k}": v for k, v in cashflow.items()}
        }
        all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_150_230 = pd.DataFrame(all_data)
df_150_230.to_csv("150_230_sp500_fundamentals.csv", index=False)
print("✅ Data saved to '150_230_sp500_fundamentals.csv'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/503] DPZ
[2/503] DASH
[3/503] DOV
[4/503] DOW
[5/503] DHI
[6/503] DTE
[7/503] DUK
[8/503] DD
[9/503] EMN
[10/503] ETN
[11/503] EBAY
[12/503] ECL
[13/503] EIX
[14/503] EW
[15/503] EA
[16/503] ELV
[17/503] EMR
[18/503] ENPH
[19/503] ETR
[20/503] EOG
[21/503] EPAM
[22/503] EQT
[23/503] EFX
[24/503] EQIX
[25/503] EQR
[26/503] ERIE
[27/503] ESS
[28/503] EL
[29/503] EG
[30/503] EVRG
[31/503] ES
[32/503] EXC
[33/503] EXE
[34/503] EXPE
[35/503] EXPD
[36/503] EXR
[37/503] XOM
[38/503] FFIV
[39/503] FDS
[40/503] FICO
[41/503] FAST
[42/503] FRT
[43/503] FDX
[44/503] FIS
[45/503] FITB
[46/503] FSLR
[47/503] FE
[48/503] FI
[49/503] F
[50/503] FTNT
[51/503] FTV
[52/503] FOXA
[53/503] FOX
[54/503] BEN
[55/503] FCX
[56/503] GRMN
[57/503] IT
[58/503] GE
[59/503] GEHC
[60/503] GEV
[61/503] GEN
[62/503] GNRC
[63/503] GD
[64/503] GIS
[65/503] GM
[66/503] GPC
[67/503] GILD
[68/503] GPN
[69/503] GL
[70/503] GDDY
[71/503] GS
[72/503] HAL
[73/503]

In [2]:
df_150_230.shape

(80, 133)

In [3]:
df_150_230.head()

,symbol,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,DPZ,2024-12-29,2024-12-29,USD,0001286681,2025-02-24,2025-02-24 06:10:31,2024,FY,4706416000,...,-532223000,-2154000,59291000,462424000,403133000,624897000,-112885000,512012000,https://www.sec.gov/Archives/edgar/data/128668...,https://www.sec.gov/Archives/edgar/data/128668...
1,DASH,2024-12-31,2024-12-31,USD,0001792789,2025-02-14,2025-02-14 16:13:32,2024,FY,10722000000,...,-204000000,-35000000,1449000000,4221000000,2772000000,2132000000,-104000000,2028000000,https://www.sec.gov/Archives/edgar/data/179278...,https://www.sec.gov/Archives/edgar/data/179278...
2,DOV,2024-12-31,2024-12-31,USD,0000029905,2025-02-14,2025-02-14 16:24:26,2024,FY,7745909000,...,-1271673000,-6348000,1429016000,1844877000,415861000,748379000,-167533000,580846000,https://www.sec.gov/Archives/edgar/data/29905/...,https://www.sec.gov/Archives/edgar/data/29905/...
3,DOW,2024-12-31,2024-12-31,USD,0001751788,2025-02-04,2025-02-04 07:57:08,2024,FY,42964000000,...,-1168000000,-163000000,-785000000,2263000000,3048000000,2914000000,-3065000000,-151000000,https://www.sec.gov/Archives/edgar/data/175178...,https://www.sec.gov/Archives/edgar/data/29915/...
4,DHI,2024-09-30,2024-09-30,USD,0000882184,2024-11-19,2024-11-19 15:47:02,2024,FY,36801400000,...,-1355300000,0,643900000,4544000000,3900100000,2189800000,-165300000,2024500000,https://www.sec.gov/Archives/edgar/data/882184...,https://www.sec.gov/Archives/edgar/data/882184...


Ho runnato solo i primi 230, poi vanno mergiati tutti insieme

# -------- PROVA CON SOLO TICKER APPLE --------

In [13]:
import pandas as pd
import requests

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
symbol = "AAPL"

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit=1&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]  # solo l'ultimo anno
    return {}

# === FETCH DATA ===
print(f"Fetching annual fundamentals for {symbol}")

income = get_fmp_data("income-statement", symbol)
balance = get_fmp_data("balance-sheet-statement", symbol)
cashflow = get_fmp_data("cash-flow-statement", symbol)

# === MERGE & CREATE DATAFRAME ===
merged = {
    "symbol": symbol,
    "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
    **income,
    **{f"balance_{k}": v for k, v in balance.items()},
    **{f"cashflow_{k}": v for k, v in cashflow.items()}
}

df_apple = pd.DataFrame([merged])  # salva in DataFrame con una riga
print(df_apple.head())  # anteprima



Fetching annual fundamentals for AAPL
  symbol  reportDate        date reportedCurrency         cik fillingDate  \
0   AAPL  2024-09-28  2024-09-28              USD  0000320193  2024-11-01   

          acceptedDate calendarYear period       revenue  ...  \
0  2024-11-01 06:01:36         2024     FY  391035000000  ...   

   cashflow_netCashUsedProvidedByFinancingActivities  \
0                                      -121983000000   

   cashflow_effectOfForexChangesOnCash  cashflow_netChangeInCash  \
0                                    0                -794000000   

   cashflow_cashAtEndOfPeriod  cashflow_cashAtBeginningOfPeriod  \
0                 29943000000                       30737000000   

   cashflow_operatingCashFlow  cashflow_capitalExpenditure  \
0                118254000000                  -9447000000   

   cashflow_freeCashFlow                                      cashflow_link  \
0           108807000000  https://www.sec.gov/Archives/edgar/data/320193...   

       

In [14]:
df_apple.columns

Index(['symbol', 'reportDate', 'date', 'reportedCurrency', 'cik',
       'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'revenue',
       ...
       'cashflow_netCashUsedProvidedByFinancingActivities',
       'cashflow_effectOfForexChangesOnCash', 'cashflow_netChangeInCash',
       'cashflow_cashAtEndOfPeriod', 'cashflow_cashAtBeginningOfPeriod',
       'cashflow_operatingCashFlow', 'cashflow_capitalExpenditure',
       'cashflow_freeCashFlow', 'cashflow_link', 'cashflow_finalLink'],
      dtype='object', length=133)

## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]